# 3D Droplet Oscillation (Part 2, Postprocessing - Aspect Ratio)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//wmg.Sessions

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
var workSess = wmg.Sessions.Skip(numSess);
workSess

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session ff159894-3ab0-4961-a743-3520554919c0 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ff159894-3ab0-4961-a743-3520554919c0\Session.info'.'


#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2*	08/01/2022 09:34:23	8e47333c...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_restart2	07/11/2022 13:39:12	95fa458f...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart2	07/11/2022 13:38:30	b97b80ef...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04*	07/11/2022 13:56:43	5480a5ac...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01*	07/11/2022 13:56:41	d98b11be...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015*	07/11/2022 13:49:48	53060394...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01_thirdOrderInit*	07/11/2022 13:49:03	325af26a...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015_thirdOrderInit*	07/11/2022 13:48:12	bc10dc66...
#9: OscillatingDroplet3D	OD3D

In [ ]:
//workSess.Pick(0).Delete(true);

In [ ]:
// var delSess = workSess.Skip(1).Take(11);
// delSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart3*	06/13/2022 15:59:18	8a39028b...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart3*	06/13/2022 15:59:17	4436c24a...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_restart1*	06/13/2022 15:59:14	e305ddba...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_restart1*	06/13/2022 15:59:13	f5d74434...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart1*	06/13/2022 15:38:06	43ae863e...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit_restart1*	06/13/2022 15:38:05	5d07f9ab...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit_restart1*	06/13/2022 15:36:54	3190027a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart3*	06/13/2022 15:36:43	8208d55e...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart3*	06/13/2022

In [ ]:
// foreach(var dS in delSess) {
//     dS.Delete(true);
// }

Session 8a39028b-afa9-4d87-8b05-4f9372ac1898 deleted.
Session 4436c24a-0e29-45ff-91ab-2d9e80e40d2e deleted.
Session e305ddba-9909-494f-88af-ec054095eb1e deleted.
Session f5d74434-16d4-4a9c-b1f5-305430715764 deleted.
Session 43ae863e-a582-4ef4-853b-e6787d8d4613 deleted.
Session 5d07f9ab-37ed-4786-9daa-959ef8271577 deleted.
During Deletion of session, trying to delete directory \\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2: System.IO.IOException: The process cannot access the file 'DropletMetrics.txt' because it is being used by another process.
Session 3190027a-c3d9-4869-bf85-ed028cbb97e2 deleted.
Session 8208d55e-f39b-4e82-a670-bdb4b37681aa deleted.
Session ecae12a5-179f-47ec-bdb0-dfb1f4f0cd06 deleted.
Session aa0a3dd8-a51e-4374-9d0c-0a35727f9214 deleted.
Session a007dfdb-2e7e-4b58-add0-224b7898c7a2 deleted.


In [ ]:
var sess = wmg.Sessions.Skip(numSess).Pick(0);
sess

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session ff159894-3ab0-4961-a743-3520554919c0 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ff159894-3ab0-4961-a743-3520554919c0\Session.info'.'


OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...

In [ ]:
sess.RestartedFrom

a4b7b06d-c801-4c5e-84ab-6aaf9f7a2e04

In [ ]:
sess.GetSessionDirectory()

\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ea13d149-d40a-48d2-88f8-28f6bfe3d38e

In [ ]:
sess.DeployPath

\\hpccluster\hpccluster-scratch\smuda\binaries\OscillatingDroplet3D-XNSE_Solver2022Jul11_134012

In [ ]:
sess.Timesteps

#0:  { Time-step: 190.0; Physical time: 0.9500000000000007s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 190.1; Physical time: 0.9500000000000007s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 190; Physical time: 0.9500000000000007s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#3:  { Time-step: 191; Physical time: 0.9550000000000007s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#4:  { Time-step: 192; Physical time: 0.9600000000000007s; Fields: Phi, PhiDG, VelocityX

In [ ]:
sess.Timesteps.Skip(0).Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit__5e0fa717-0453-4cc2-a269-5dec740905d0


In [ ]:
using System.IO;

# Study Overview

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01", "m3_Oh01_eta04", "m3_Oh01_eta03", "m3_Oh01_eta015", "m4_Oh01_eta04", "m4_Oh01_eta01", "m4_Oh056_eta005"};
//string[] caseS = new string[] { "m2_Oh01_eta04", "m3_Oh01_eta04", "m3_Oh01_eta015", "m4_Oh01_eta01"};
string[] gridS = new string[] { "J686" }; //, "J432" };
int[] amrS = new int[] { 1, 2 };
bool[] ShouldAnaInit = new bool[] { false, true };
bool[] useNewton = new bool[] { false };

In [ ]:
var studySess = new List<ISessionInfo>();
foreach(string _case in caseS) {
    foreach(int _amr in amrS) {
        foreach(bool _init in ShouldAnaInit) {
            foreach(var sess in workSess) {
                if(sess.Name.Contains(_case) && sess.Name.Contains("amr"+_amr) && sess.Name.Contains(gridS[0])) {
                    if(!(_init ^ sess.Name.Contains("_thirdOrderInit"))) {
                        studySess.Add(sess);
                        break;
                    }
                }
            }
        }
    }
}

Loading session 8e47333c-073c-4b3d-8c88-e717cf2fc784 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e47333c-073c-4b3d-8c88-e717cf2fc784\Session.info'.'
Loading session 8e47333c-073c-4b3d-8c88-e717cf2fc784 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e47333c-073c-4b3d-8c88-e717cf2fc784\Session.info'.'
Loading session 8e47333c-073c-4b3d-8c88-e717cf2fc784 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e47333c-073c-4b3d-8c88-e717cf2fc784\Session.info'.'
Loading session 8e47333c-073c-4b3d-8c88-e717cf2fc784 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e47333c-073c-4b3d-8c88-e717cf2fc784\Session.info'.'
Loading session 8e47333c-073c-4b3d-8c88-e717cf2fc784 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\Osc

In [ ]:
//var studySess = workSess.Skip(3).Take(18);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart3	06/13/2022 15:31:38	ceb7f024...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart3	06/15/2022 08:37:47	95fc59fd...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart3	06/15/2022 09:32:54	d8e3f635...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart3	06/15/2022 08:37:41	dd97e2f1...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart3	06/15/2022 08:37:38	0d35c433...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart2	07/11/2022 13:38:30	b97b80ef...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit*	05/30/2022 14:36:38	4f6ae21b...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04*	07/11/2022 13:56:43	5480a5ac...
#9: Osci

In [ ]:
// string oldName = studySess.Pick(0).Name;
// oldName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart1_restart2

In [ ]:
// string  newName = oldName.Substring(0, oldName.Length - 10) + '2';
// newName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2

In [ ]:
// studySess.Pick(0).Name = newName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
// Plot2Ddata[,] PlotTableDM_unprocessed = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAMR = 0; iAMR < amrS.Length; iAMR++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
//for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];
    int _amr = amrS[iAMR];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && sess.Name.Contains("amr"+_amr)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna]));
                                      //&& (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);


    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    //Console.WriteLine("number of processed Sessions for {0}: {1}", _case, procSIs.Count);

    var plot = new Plot2Ddata();
    // var plot2 = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.DashType = DashTypes.Solid;
    fmt.LineWidth = 2;
    // if (useNewton[iNew]) {
    //     fmt.DashType = DashTypes.Dashed;
    // } else {  
    //     fmt.DashType = DashTypes.Solid;
    // }
    if (ShouldAnaInit[iAna]) {
        fmt.LineColor= LineColors.Red;
    } else {
        fmt.LineColor= LineColors.Blue;
    }

    if(_amr == 2) 
        fmt.DashType = DashTypes.DotDashed; 
    
 
    string name = "BoSSS-" + _grd + "-AMR" + _amr;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    // if(useNewton[iNew])
    //     name = name + "-Newton";


    double[] time = new double[0];
    double[] arLW = new double[0];

    double[] arL = new double[0];
    double[] arWx = new double[0];
    double[] arWy = new double[0];

    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session: {0}", pSI.Name);
        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }
        
        time = time.Concat(tab["time"]).ToArray();

        double[] L = tab["L"].ToArray(); 
        double[] Wx = tab["Wx"].ToArray(); 
        double[] Wy = tab["Wy"].ToArray();
        double[] LW = new double[L.Length];
        for (int i = 0; i < L.Length; i++) {
            LW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
        }
        arLW = arLW.Concat(LW).ToArray();

        arL = arL.Concat(L).ToArray();
        arWx = arWx.Concat(Wx).ToArray();
        arWy = arWy.Concat(Wy).ToArray();

    }

    // plot2.AddDataGroup("L", time, arL, fmt);
    // plot2.AddDataGroup("Wx", time, arWx, fmt);
    // plot2.AddDataGroup("Wy", time, arWy, fmt);

    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.01;
    for(int i = 0; i < time.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
        // PlotTableDM_unprocessed[iRow,iCol] = plot2;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
        // PlotTableDM_unprocessed[iRow,iCol] = PlotTableDM_unprocessed[iRow,iCol].Merge(plot2);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}
//}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart3
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_restart2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_restart2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015
OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart3
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart3
OD3D_J686k3

In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta04 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta02 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta01 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta04 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta03 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta015 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta04 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta01 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh056_eta005

### Period durations

In [ ]:
Plot2Ddata[,] PlotTablePD = new Plot2Ddata[caseS.Length, 1];
for(int iRow = 3; iRow < 4; iRow++) { // loop over cases
    Plot2Ddata plot = PlotTableDM[iRow,0];
    double[] time = plot.dataGroups[0].Abscissas;
    double[] LW = plot.dataGroups[0].Values;

    List<double> times_max = new List<double>();    // times at maxima
    List<double> times_min = new List<double>();    // times at minima
    List<double> times_one = new List<double>();    // times at value 1 crossings
    double value_max = LW[0];
    double value_min = double.MaxValue;
    double time_maxmin = 0;
    bool search_max = true;     // if false the local minima is searched for
    for(int i = 1; i < time.Length; i++) {
        double currVal = LW[i];
        if(search_max && currVal > value_max) {
            value_max = currVal;
            time_maxmin = time[i];
        } else if(!search_max && currVal < value_min) {
            value_min = currVal;
            time_maxmin = time[i];
        }
        double prevVal = LW[i-1];
        if(prevVal >= 1 && currVal < 1) {   // value 1 neg crossing
            times_one.Add(time[i]);
            times_max.Add(time_maxmin);
            value_max = 1.0;
            search_max = false;
        } else if(prevVal < 1 && currVal >= 1) {   // value 1 pos crossing
            times_one.Add(time[i]);
            times_min.Add(time_maxmin);
            value_min = 1.0;
            search_max = true;
        }
    }

    List<double> periodTimes = new List<double>();
    List<double> timesAtPstart = new List<double>(); 

    for(int i = 0; i < times_max.Count - 1; i++) {
        periodTimes.Add(times_max.ElementAt(i+1) - times_max.ElementAt(i)); 
        timesAtPstart.Add(times_max.ElementAt(i));
    }
    for(int i = 0; i < times_min.Count - 1; i++) {
        periodTimes.Add(times_min.ElementAt(i+1) - times_min.ElementAt(i)); 
        timesAtPstart.Add(times_min.ElementAt(i));
    }
    for(int i = 0; i < times_one.Count - 2; i++) {
        periodTimes.Add(times_one.ElementAt(i+2) - times_one.ElementAt(i)); 
        timesAtPstart.Add(times_one.ElementAt(i));
    }

    Plot2Ddata procPlot = new Plot2Ddata();
    var fmt = new PlotFormat();
    fmt.Style = Styles.Points;
    fmt.PointSize = 1;
    procPlot.AddDataGroup(plot.dataGroups[0].Name, timesAtPstart, periodTimes, fmt);

    PlotTablePD[iRow, 0] = procPlot;
}


## Case 1 (m2_Oh01_eta04)

In [ ]:
var pltTab1 = PlotTableDM[0,0];
pltTab1.Title

m2_Oh01_eta04

In [ ]:
//var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/Oh_0dot1_LWinTime.txt");
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta04.txt");
var basaran_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/basaranOscOh_0dot1.txt");
var becker_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/beckerEtAlOscOh_0dot1.txt");
var meradji_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/meradjiOscOh_0dot1.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
pltTab1.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 1;
fmt.LineColor = LineColors.Magenta;
pltTab1.AddDataGroup("Basaran (1992)", basaran_dat.GetColumn(0), basaran_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 1;
fmt.LineColor = LineColors.Green;
pltTab1.AddDataGroup("Becker et. al. (1994)", becker_dat.GetColumn(0), becker_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 1;
fmt.LineColor = LineColors.Orange;
pltTab1.AddDataGroup("Meradji et. al. (2001)", meradji_dat.GetColumn(0), meradji_dat.GetColumn(1), fmt);

//var gp = dat_plot.ToGnuplot();
//gp.PlotSVG()

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 7;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta04 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M71.9,207.1 L72.4,207.1 L72.9,207.2 L73.4,207.3 L73.8,207.5 L74.3,207.7
 L74.8,207.9 L75.3,208.2 L75.8,208.6 L76.3,209.0 L76.7,209.4 L77.2,209.8 L77.7,210.2 L78.2,210.7
 L78.7,211.2 L79.2,211.7 L79.7,212.3 L80.1,212.8 L80.6,213.4 L81.1,214.0 L81.6,214.6 L82.1,215.2
 L82.6,215.8 L83.0,216.4 L83.5,217.1 L84.0,217.7 L84.5,218.4 L85.0,219.0 L85.5,219.7 L85.9,220.4
 L86.4,221.1 L86.9,221.8 L87.4,222.5 L87.9,223.2 L88.4,223.9 L88.9,224.7 L89.3,225.4 L89.8,226.1
 L90.3,226.9 L90.8,227.6 L91.3,228.4 L91.8,229.1 L92.2,229.9 L92.7,230.6 L93.2,231.4 L93.7,232.2
 L94.2,232.9 L94.7,233.7 L95.2,234.5 L95.6,235.2 L96.1,236.0 L96.6,236.8 L97.1,237.6 L97.6,238.4
 L98.1,239.2 L98.5,240.0 L99.0,240.8 L99.5,241.6 L100.0,242.4 L100.5,243.2 L101.0,244.0 L101.5,244.9
 L101.9,245.7 L102.4,246.5 L102.9,247.3 L103.4,248.2 L103.9,249.0 L104.4,249.8 L104.8,250.7 L105.3,251.6
 L105.8,252.5 L106.3,253.3 L106.8,254.2 L107.3,255.0 L107.7,255.9 L108.2,256.7 L108.7,257.6 L109.2,258.5
 L109.7,259.4 L110.2,260.3 L110.7,261.2 L111.1,262.1 L111.6,263.0 L112.1,263.9 L112.6,264.8 L113.1,265.8
 L113.6,266.7 L114.0,267.6 L114.5,268.6 L115.0,269.6 L115.5,270.6 L116.0,271.6 L116.5,272.6 L117.0,273.7
 L117.4,274.6 L117.9,275.7 L118.4,276.7 L118.9,277.8 L119.4,278.8 L119.9,279.8 L120.3,280.9 L120.8,282.0
 L121.3,283.1 L121.8,284.2 L122.3,285.4 L122.8,286.5 L123.2,287.6 L123.7,288.8 L124.2,289.9 L124.7,291.1
 L125.2,292.2 L125.7,293.4 L126.2,294.6 L126.6,295.8 L127.1,297.0 L127.6,298.2 L128.1,299.4 L128.6,300.7
 L129.1,301.9 L129.5,303.1 L130.0,304.4 L130.5,305.7 L131.0,306.9 L131.5,308.2 L132.0,309.5 L132.5,310.8
 L132.9,312.0 L133.4,313.3 L133.9,314.6 L134.4,315.9 L134.9,317.2 L135.4,318.5 L135.8,319.8 L136.3,321.1
 L136.8,322.4 L137.3,323.7 L137.8,325.0 L138.3,326.3 L138.8,327.6 L139.2,328.9 L139.7,330.1 L140.2,331.4
 L140.7,332.7 L141.2,334.0 L141.7,335.2 L142.1,336.5 L142.6,337.8 L143.1,339.0 L143.6,340.3 L144.1,341.5
 L144.6,342.8 L145.0,344.0 L145.5,345.3 L146.0,346.5 L146.5,347.7 L147.0,348.9 L147.5,350.1 L148.0,351.3
 L148.4,352.5 L148.9,353.7 L149.4,354.9 L149.9,356.0 L150.4,357.2 L150.9,358.3 L151.3,359.4 L151.8,360.5
 L152.3,361.6 L152.8,362.7 L153.3,363.8 L153.8,364.9 L154.3,366.0 L154.7,367.0 L155.2,368.1 L155.7,369.1
 L156.2,370.2 L156.7,371.2 L157.2,372.1 L157.6,373.1 L158.1,374.1 L158.6,375.1 L159.1,376.0 L160.1,377.8
 L160.6,378.7 L161.0,379.6 L161.5,380.5 L162.0,381.3 L162.5,382.1 L163.0,382.9 L163.5,383.7 L163.9,384.5
 L164.4,385.3 L164.9,386.1 L165.4,386.9 L165.9,387.7 L166.4,388.4 L166.8,389.2 L167.3,389.9 L167.8,390.6
 L168.3,391.3 L168.8,392.0 L169.3,392.5 L169.8,393.1 L170.2,393.7 L170.7,394.2 L171.2,394.8 L171.7,395.3
 L172.2,395.8 L172.7,396.3 L173.1,396.8 L173.6,397.4 L174.1,397.8 L174.6,398.3 L175.1,398.8 L175.6,399.3
 L176.1,399.7 L176.5,400.1 L177.0,400.6 L177.5,400.9 L178.0,401.4 L178.5,401.7 L179.0,402.1 L179.4,402.5
 L179.9,402.8 L180.4,403.1 L180.9,403.5 L181.4,403.8 L181.9,404.0 L182.3,404.3 L182.8,404.6 L183.3,404.8
 L183.8,405.1 L184.3,405.3 L184.8,405.5 L185.3,405.7 L185.7,405.9 L186.2,406.1 L186.7,406.3 L187.2,406.4
 L187.7,406.6 L188.2,406.7 L188.6,406.8 L189.1,407.0 L189.6,407.1 L190.1,407.1 L190.6,407.2 L191.1,407.3
 L191.6,407.4 L192.0,407.4 L192.5,407.4 L193.0,407.5 L193.5

In [ ]:
foreach(var grp in pltTab1.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 2 (m2_Oh01_eta02)

In [ ]:
var pltTab2 = PlotTableDM[1,0];
pltTab2.Title

m2_Oh01_eta02

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta02.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
PlotTableDM[1,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab2.XrangeMin = 0;
pltTab2.XrangeMax = 7;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta02 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M71.9,122.0 L72.4,122.0 L72.9,122.1 L73.4,122.2 L73.8,122.3 L74.3,122.5
 L74.8,122.8 L75.3,123.1 L75.8,123.4 L76.3,123.8 L76.7,124.3 L77.2,124.7 L77.7,125.3 L78.2,125.8
 L78.7,126.5 L79.2,127.1 L79.7,127.8 L80.1,128.5 L80.6,129.3 L81.1,130.1 L81.6,130.9 L82.1,131.8
 L82.6,132.7 L83.0,133.6 L83.5,134.6 L84.0,135.6 L84.5,136.6 L85.0,137.7 L85.5,138.8 L85.9,139.9
 L86.4,141.1 L86.9,142.3 L87.4,143.5 L87.9,144.7 L88.4,146.0 L88.9,147.3 L89.3,148.6 L89.8,149.9
 L90.3,151.3 L90.8,152.7 L91.3,154.1 L91.8,155.6 L92.2,157.1 L92.7,158.5 L93.2,160.1 L93.7,161.6
 L94.2,163.2 L94.7,164.8 L95.2,166.4 L95.6,168.0 L96.1,169.7 L96.6,171.4 L97.1,173.1 L97.6,174.8
 L98.1,176.6 L98.5,178.4 L99.0,180.2 L99.5,182.0 L100.0,183.8 L100.5,185.7 L101.0,187.6 L101.5,189.5
 L101.9,191.4 L102.4,193.4 L102.9,195.4 L103.4,197.3 L103.9,199.4 L104.4,201.4 L104.8,203.5 L105.3,205.6
 L105.8,207.7 L106.3,209.9 L106.8,212.1 L107.3,214.2 L107.7,216.4 L108.2,218.7 L108.7,220.9 L109.2,223.2
 L109.7,225.5 L110.2,227.7 L110.7,230.0 L111.1,232.4 L111.6,234.8 L112.1,237.2 L112.6,239.6 L113.1,242.0
 L113.6,244.5 L114.0,246.9 L114.5,249.4 L115.0,252.0 L115.5,254.5 L116.0,257.0 L116.5,259.6 L117.0,262.2
 L117.4,264.8 L117.9,267.2 L118.4,269.9 L118.9,272.6 L119.4,275.1 L119.9,277.8 L120.3,280.5 L120.8,283.2
 L121.3,285.9 L121.8,288.5 L122.3,291.2 L122.8,294.0 L123.2,296.7 L123.7,299.3 L124.2,302.1 L124.7,304.9
 L125.2,307.7 L125.7,310.3 L126.2,313.1 L126.6,315.9 L127.1,318.7 L127.6,321.4 L128.1,324.2 L128.6,326.9
 L129.1,329.6 L129.5,332.4 L130.0,335.2 L130.5,338.0 L131.0,340.7 L131.5,343.5 L132.0,346.2 L132.5,349.0
 L132.9,351.7 L133.4,354.4 L133.9,357.1 L134.4,359.8 L134.9,362.5 L135.4,365.2 L135.8,367.9 L136.3,370.5
 L136.8,373.2 L137.3,375.8 L137.8,378.4 L138.3,381.0 L138.8,383.6 L139.2,386.1 L139.7,388.7 L140.2,391.2
 L140.7,393.7 L141.2,396.2 L141.7,398.7 L142.1,401.2 L142.6,403.6 L143.1,406.1 L143.6,408.5 L144.1,410.8
 L144.6,413.2 L145.0,415.6 L145.5,417.9 L146.0,420.2 L146.5,422.5 L147.0,424.7 L147.5,426.9 L148.0,429.1
 L148.4,431.4 L148.9,433.5 L149.4,435.7 L149.9,437.8 L150.4,440.0 L150.9,442.0 L151.3,444.1 L151.8,446.1
 L152.3,448.1 L152.8,450.1 L153.3,452.0 L153.8,453.9 L154.3,455.8 L154.7,457.5 L155.2,459.4 L155.7,461.2
 L156.2,463.0 L156.7,464.7 L157.2,466.4 L157.6,468.1 L158.1,469.8 L158.6,471.4 L159.1,473.0 L159.6,474.5
 L160.1,476.1 L160.6,477.6 L161.0,479.1 L161.5,480.5 L162.0,481.9 L162.5,483.3 L163.0,484.7 L163.5,486.0
 L163.9,487.3 L164.4,488.6 L164.9,489.8 L165.4,491.0 L165.9,492.2 L166.4,493.4 L171.7,504.0 L172.2,504.8
 L172.7,505.7 L173.1,506.5 L173.6,507.1 L174.1,507.9 L174.6,508.6 L175.1,509.2 L175.6,509.9 L176.1,510.4
 L176.5,511.0 L177.0,511.5 L177.5,512.0 L178.0,512.5 L178.5,512.9 L179.0,513.3 L179.4,513.7 L179.9,514.1
 L180.4,514.4 L180.9,514.7 L181.4,515.0 L181.9,515.3 L191.1,515.0 L191.6,514.8 L192.0,514.5 L192.5,514.2
 L193.0,513.8 L193.5,513.5 L194.0,513.1 L194.5,512.7 L194.9,512.2 L195.4,511.8 L195.9,511.3 L196.4,510.8
 L196.9,510.3 L197.4,509.8 L197.9,509.2 L198.3,508.6 L198.8,508.0 L199.3,507.4 L199.8,506.7 L200.3,505.9
 L200.8,505.2 L201.2,504.5 L201.7,503.8 L202.2,503.1 L202.7,502.3 L203.2,501.5 L203.7,500.7 L204.1,499.9
 L204.6,499.0 L205.1,498.1 L205

In [ ]:
foreach(var grp in pltTab2.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m2/" + pltTab2.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 3 (m2_Oh01_eta01)

In [ ]:
var pltTab3 = PlotTableDM[2,0];
pltTab3.Title

m2_Oh01_eta01

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta01.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab3.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 7;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta01 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M80.2,137.5 L80.7,137.5 L81.2,137.6 L81.6,137.7 L82.1,137.8 L82.6,138.0
 L83.1,138.2 L83.5,138.5 L84.0,138.8 L84.5,139.1 L85.0,139.5 L85.5,140.0 L85.9,140.4 L86.4,140.9
 L86.9,141.5 L87.4,142.1 L87.9,142.7 L88.3,143.4 L88.8,144.1 L89.3,144.8 L89.8,145.6 L90.2,146.5
 L90.7,147.3 L91.2,148.2 L91.7,149.1 L92.2,150.1 L92.6,151.1 L93.1,152.2 L93.6,153.2 L94.1,154.4
 L94.6,155.5 L95.0,156.7 L95.5,157.9 L96.0,159.2 L96.5,160.4 L96.9,161.8 L97.4,163.1 L97.9,164.5
 L98.4,165.9 L98.9,167.4 L99.3,168.8 L99.8,170.4 L100.3,171.9 L100.8,173.5 L101.3,175.1 L101.7,176.7
 L102.2,178.3 L102.7,180.0 L103.2,181.8 L103.6,183.5 L104.1,185.3 L104.6,187.1 L105.1,189.0 L105.6,190.8
 L106.0,192.7 L106.5,194.6 L107.0,196.6 L107.5,198.5 L108.0,200.5 L108.4,202.6 L108.9,204.6 L109.4,206.7
 L109.9,208.8 L110.3,210.9 L110.8,213.0 L111.3,215.2 L111.8,217.5 L112.3,219.7 L112.7,221.9 L113.2,224.2
 L113.7,226.5 L114.2,228.8 L114.7,231.2 L115.1,233.5 L115.6,235.9 L116.1,238.3 L116.6,240.7 L117.0,243.2
 L117.5,245.6 L118.0,248.1 L118.5,250.5 L119.0,253.0 L119.4,255.5 L119.9,258.1 L120.4,260.6 L120.9,263.2
 L121.4,265.8 L121.8,268.4 L122.3,271.0 L122.8,273.6 L123.3,276.3 L123.7,278.9 L124.2,281.5 L124.7,284.2
 L125.2,286.9 L125.7,289.5 L126.1,292.2 L126.6,294.9 L127.1,297.6 L127.6,300.3 L128.1,303.0 L128.5,305.8
 L129.0,308.5 L129.5,311.2 L130.0,314.0 L130.4,316.7 L130.9,319.4 L131.4,322.2 L131.9,324.9 L132.4,327.7
 L132.8,330.4 L133.3,333.1 L133.8,335.9 L134.3,338.6 L134.7,341.3 L135.2,344.1 L135.7,346.8 L136.2,349.5
 L136.7,352.2 L137.1,354.9 L137.6,357.6 L138.1,360.3 L138.6,363.0 L139.1,365.7 L139.5,368.3 L140.0,371.0
 L140.5,373.6 L141.0,376.3 L141.4,378.9 L141.9,381.5 L142.4,384.1 L142.9,386.7 L143.4,389.3 L143.8,391.8
 L144.3,394.4 L144.8,396.9 L145.3,399.4 L145.8,401.9 L146.2,404.4 L146.7,406.9 L147.2,409.3 L147.7,411.8
 L148.1,414.2 L148.6,416.6 L149.1,419.0 L149.6,421.3 L150.1,423.7 L150.5,426.0 L151.0,428.4 L151.5,430.7
 L152.0,432.9 L152.5,435.1 L152.9,437.3 L153.4,439.5 L153.9,441.6 L154.4,443.8 L154.8,445.9 L155.3,448.0
 L155.8,450.1 L156.3,452.1 L156.8,454.1 L157.2,456.2 L157.7,458.2 L158.2,460.1 L158.7,462.0 L159.2,463.9
 L159.6,465.8 L160.1,467.7 L160.6,469.5 L161.1,471.4 L161.5,473.1 L162.0,474.9 L162.5,476.6 L163.0,478.3
 L163.5,480.0 L163.9,481.6 L164.4,483.2 L164.9,484.8 L165.4,486.4 L165.9,487.9 L166.3,489.4 L166.8,490.8
 L167.3,492.3 L167.8,493.7 L168.2,495.1 L168.7,496.4 L169.2,497.7 L169.7,499.0 L170.2,500.3 L170.6,501.5
 L171.1,502.7 L171.6,503.9 L172.1,505.1 L172.6,506.2 L173.0,507.3 L173.5,508.3 L174.0,509.3 L174.5,510.3
 L174.9,511.3 L175.4,512.3 L175.9,513.2 L176.4,514.0 L176.9,514.9 L177.3,515.7 L177.8,516.5 L178.3,517.3
 L178.8,518.0 L179.2,518.7 L179.7,519.4 L180.2,520.0 L180.7,520.6 L181.2,521.2 L181.6,521.8 L182.1,522.1
 L182.6,522.7 L183.1,523.2 L183.6,523.6 L184.0,524.1 L184.5,524.5 L185.0,524.8 L185.5,525.2 L185.9,525.5
 L186.4,525.7 L186.9,526.0 L187.4,526.2 L187.9,526.4 L188.3,526.5 L188.8,526.6 L189.3,526.7 L189.8,526.8
 L190.3,526.8 L190.7,526.8 L191.2,526.8 L191.7,526.8 L192.2,526.7 L192.6,526.6 L193.1,526.5 L193.6,526.3
 L194.1,526.1 L194.6,525.9 L195.0,525.7 L195.5,525.4 L196.0,525.2 L196.5,524.8 L197.0,524.5 L197.4,524.1
 L197.9,523.

In [ ]:
foreach(var grp in pltTab3.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m2/" + pltTab3.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 4 (m3_Oh01_eta04)

In [ ]:
var pltTab4 = PlotTableDM[3,0];
pltTab4.Title

m3_Oh01_eta04

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m3/AspRatioInTime_m3_Oh01_eta04.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab4.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab4.XrangeMin = 0;
pltTab4.XrangeMax = 1;
var gp = pltTab4.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.98 
 
 
 
 
 1 
 
 
 
 
 1.02 
 
 
 
 
 1.04 
 
 
 
 
 1.06 
 
 
 
 
 1.08 
 
 
 
 
 1.1 
 
 
 
 
 1.12 
 
 
 
 
 1.14 
 
 
 
 
 1.16 
 
 
 
 
 1.18 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta04 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M80.2,497.1 L83.5,497.6 L86.9,494.9 L90.2,490.9 L93.6,485.5 L96.9,479.3
 L100.3,472.0 L103.6,464.1 L107.0,455.8 L110.3,447.3 L113.7,438.8 L117.0,430.5 L120.4,421.4 L123.7,412.1
 L127.1,403.4 L130.4,394.2 L133.8,382.7 L137.1,374.2 L140.5,366.1 L143.8,358.4 L147.2,350.9 L150.5,343.8
 L153.9,335.7 L157.2,326.9 L160.6,321.3 L163.9,314.6 L167.3,307.3 L170.6,300.4 L174.0,293.3 L177.3,287.2
 L180.7,280.3 L184.0,287.4 L187.4,281.6 L190.7,276.0 L194.1,269.8 L197.4,264.3 L200.8,257.9 L204.1,252.8
 L207.5,247.6 L210.8,242.3 L214.2,237.2 L217.5,233.6 L220.9,226.3 L224.2,219.2 L227.6,224.6 L230.9,215.9
 L234.3,206.6 L237.6,199.4 L241.0,193.6 L244.3,190.0 L247.7,185.6 L251.0,181.7 L254.4,178.4 L257.7,175.6
 L261.1,172.7 L264.4,171.0 L267.8,168.9 L271.1,169.6 L274.5,167.6 L277.8,165.7 L281.2,163.9 L284.5,160.7
 L287.9,159.2 L291.2,159.2 L294.6,157.7 L297.9,156.4 L301.3,153.3 L304.6,153.4 L308.0,151.7 L311.3,151.3
 L314.7,152.6 L318.0,151.4 L321.4,151.4 L324.7,153.2 L328.1,153.7 L331.4,153.1 L334.8,153.1 L338.1,154.9
 L341.5,155.1 L344.8,158.1 L348.2,157.5 L351.5,159.8 L354.9,162.4 L358.2,165.8 L361.6,168.9 L364.9,173.7
 L368.3,179.2 L371.6,181.0 L375.0,184.4 L378.3,186.3 L381.7,192.3 L385.0,194.2 L388.4,194.9 L391.7,198.3
 L395.1,202.1 L398.4,206.2 L401.8,210.5 L405.1,215.6 L408.5,219.8 L411.8,224.5 L415.2,228.8 L418.5,233.1
 L421.8,237.5 L425.2,241.8 L428.5,245.8 L431.9,249.7 L435.2,253.5 L438.6,256.0 L441.9,259.5 L445.3,263.4
 L448.6,267.1 L452.0,270.4 L455.3,273.6 L458.7,275.3 L462.0,507.9 L465.4,280.6 L468.7,283.3 L472.1,285.1
 L475.4,287.3 L478.8,282.2 L482.1,283.7 L485.5,284.9 L488.8,285.9 L492.2,296.6 L495.5,289.3 L498.9,289.8
 L502.2,297.7 L505.6,289.7 L508.9,289.6 L512.3,289.5 L515.6,289.1 L519.0,288.7 L522.3,288.1 L525.7,287.4
 L529.0,286.8 L532.4,286.1 L535.7,285.4 L539.1,284.6 L542.4,289.5 L545.8,288.8 L549.1,288.2 L552.5,287.4
 L555.8,286.9 L559.2,286.2 L562.5,285.3 L565.9,284.4 L569.2,283.3 L572.6,282.2 L575.9,281.0 L579.3,279.8
 L582.6,278.6 L586.0,277.5 L589.3,274.0 L592.7,274.5 L596.0,272.1 L599.4,271.3 L602.7,272.4 L606.1,272.3
 L609.4,271.7 L612.8,270.7 L616.1,269.7 L619.5,268.6 L622.8,267.7 L626.2,263.1 L629.5,264.4 L632.9,264.8
 L636.2,262.2 L639.6,260.1 L642.9,262.4 L646.3,263.3 L649.6,264.6 L653.0,264.5 L656.3,264.2 L659.7,263.8
 L663.0,263.5 L666.4,263.2 L669.7,269.0 L673.1,269.1 L676.4,269.3 L679.8,269.7 L683.1,270.7 L686.5,273.1
 L689.8,274.7 L693.2,277.8 L696.5,279.1 L699.9,280.5 L703.2,282.0 L706.6,283.6 L709.9,285.3 L713.3,287.1
 L716.6,288.9 L720.0,291.7 L723.3,292.3 L726.7,381.5 L730.0,297.0 L733.4,297.8 L736.7,298.7 L740.1,300.1
 L743.4,301.3 L746.8,302.4 L750.1,304.3 '/> 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 
 
 
 BoSSS-J686-AMR2 
 
 
 BoSSS-J686-AMR2 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray=' 8,4,2,4' d='M410.9,141.1 L464.3,141.1 M80.2,497.1 L81.9,492.2 L83.5,491.7 L85.2,490.7 L86.9,488.5 L88.6,486.6
 L90.2,484.2 L91.9,481.0 L93.6,477.9 L95.3,474.7 L96.9,471.3 L98.6,467.8 L100.3,464.0 L102.0,459.8
 L103.6,455.8 L105.3,451.7 L107.0,447.4 L108.7,442.9 L110.3,438.9 L112.0,434.7 L113.7,430.2 L115.4,426.0
 L117.0,421.7 

In [ ]:
foreach(var grp in pltTab4.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m3/" + pltTab4.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Checking data

In [ ]:
// var pltTab4_unproc = PlotTableDM_unprocessed[3,0];

// var format = pltTab4_unproc.dataGroups[0].Format.CloneAs();
// format.LineColor = LineColors.Red;
// pltTab4_unproc.dataGroups[1].Format = format;

// format = pltTab4_unproc.dataGroups[0].Format.CloneAs();
// format.LineColor = LineColors.Orange;
// pltTab4_unproc.dataGroups[2].Format = format;

In [ ]:
// Plot2Ddata plt = new Plot2Ddata();

// plt.AddDataGroup(pltTab4_unproc.dataGroups[0]);

// plt.XrangeMin = 0;
// plt.XrangeMax = 2;
// var gp = plt.ToGnuplot();
// gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9 
 
 
 
 
 1.95 
 
 
 
 
 2 
 
 
 
 
 2.05 
 
 
 
 
 2.1 
 
 
 
 
 2.15 
 
 
 
 
 2.2 
 
 
 
 
 2.25 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 
 
 
 
 
 L 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M666.2,57.1 L719.6,57.1 M62.2,480.8 L63.9,481.4 L65.6,478.1 L67.4,473.3 L69.1,466.7 L70.8,459.2
 L72.5,450.4 L74.2,440.8 L76.0,430.8 L77.7,420.7 L79.4,410.6 L81.1,400.7 L82.8,390.0 L84.6,379.1
 L86.3,368.9 L88.0,358.3 L89.7,344.7 L91.4,335.1 L93.2,325.9 L94.9,317.1 L96.6,308.8 L98.3,300.8
 L100.0,291.8 L101.8,283.4 L103.5,277.2 L105.2,269.9 L106.9,262.0 L108.6,254.1 L110.4,246.3 L112.1,239.4
 L113.8,231.7 L115.5,224.3 L117.2,216.6 L119.0,209.0 L120.7,200.7 L122.4,193.2 L124.1,184.7 L125.8,177.6
 L127.6,170.4 L129.3,163.3 L131.0,154.8 L132.7,150.5 L134.4,141.4 L136.1,132.8 L137.9,139.7 L139.6,128.8
 L141.3,117.2 L143.0,108.1 L144.7,100.8 L146.5,96.3 L148.2,90.9 L149.9,85.9 L151.6,81.6 L153.3,77.9
 L155.1,74.3 L156.8,72.1 L158.5,69.6 L160.2,67.1 L161.9,65.1 L163.7,63.2 L165.4,61.4 L167.1,59.7
 L168.8,58.1 L170.5,56.7 L172.3,55.4 L174.0,54.4 L175.7,51.7 L177.4,52.2 L179.1,50.8 L180.9,50.8
 L182.6,52.9 L184.3,52.1 L186.0,52.8 L187.7,55.7 L189.5,57.1 L191.2,57.5 L192.9,58.6 L194.6,62.0
 L196.3,63.8 L198.1,68.8 L199.8,69.4 L201.5,73.6 L203.2,78.3 L204.9,83.8 L206.7,89.0 L208.4,96.5
 L210.1,104.7 L211.8,108.2 L213.5,113.7 L215.3,117.5 L217.0,126.2 L218.7,129.9 L220.4,135.1 L222.1,140.8
 L223.9,147.0 L225.6,153.6 L227.3,160.5 L229.0,167.4 L230.7,174.0 L232.5,181.0 L234.2,187.6 L235.9,194.2
 L237.6,200.8 L239.3,207.1 L241.1,213.1 L242.8,219.0 L244.5,224.6 L246.2,228.7 L247.9,234.0 L249.7,239.8
 L251.4,245.1 L253.1,250.0 L254.8,254.6 L256.5,257.4 L258.3,545.0 L260.0,265.2 L261.7,269.2 L263.4,271.7
 L265.1,275.0 L266.9,277.6 L268.6,279.9 L270.3,281.8 L272.0,283.4 L273.7,287.3 L275.4,288.2 L277.2,288.9
 L278.9,288.8 L280.6,288.9 L282.3,288.8 L284.0,288.5 L285.8,287.9 L287.5,287.0 L289.2,286.0 L290.9,284.8
 L292.6,283.4 L294.4,282.1 L296.1,280.6 L297.8,278.9 L299.5,274.2 L301.2,272.8 L303.0,271.4 L304.7,269.5
 L306.4,268.3 L308.1,266.6 L309.8,264.6 L311.6,262.4 L313.3,260.1 L315.0,257.6 L316.7,255.0 L318.4,252.4
 L320.2,249.8 L321.9,247.1 L323.6,241.4 L325.3,240.9 L327.0,236.5 L328.8,234.3 L330.5,234.5 L332.2,233.0
 L333.9,230.9 L335.6,228.3 L337.4,225.6 L339.1,222.8 L340.8,220.1 L342.5,212.7 L344.2,213.0 L346.0,212.1
 L347.7,207.0 L349.4,205.4 L351.1,204.1 L352.8,204.1 L354.6,204.5 L356.3,202.9 L358.0,201.1 L359.7,199.1
 L361.4,197.1 L363.2,195.3 L364.9,197.5 L366.6,196.4 L368.3,195.4 L370.0,194.6 L371.8,194.8 L373.5,196.7
 L375.2,197.6 L376.9,200.3 L378.6,200.9 L380.4,201.6 L382.1,202.3 L383.8,203.2 L385.5,204.2 L387.2,205.4
 L389.0,206.7 L390.7,209.6 L392.4,209.5 L394.1,320.9 L395.8,213.4 L397.6,213.5 L399.3,213.8 L401.0,214.8
 L402.7,215.5 L404.4,216.0 L406.2,217.8 L407.9,218.6 L409.6,227.9 L411.3,230.9 L413.0,241.6 L414.7,249.6
 L416.5,258.1 L418.2,264.1 L419.9,267.0 L421.6,268.2 L423.3,270.2 L425.1,271.3 L426.8,272.3 L428.5,273.5
 L430.2,274.8 L431.9,275.3 L433.7,277.1 L435.4,280.5 L437.1,281.9 L438.8,284.6 L440.5,287.5 L442.3,290.6
 L444.0,293.8 L445.7,296.6 L447.4,300.0 L449.1,303.8 L450.9,306.0 L452.6,310.4 L454.3,314.4 L456.0,317.6
 L457.7,321.1 L459.5,324.4 L461.2,327.6 L462.9,330.6 L464.6,333.6 L466.3,339.3 L468.1,342.5 L469.8,345.8
 L471.5,349.0 L473.2,352.2 L474.9,355.4 L476.7,358.6 L478.4,364.2 L480.1,365.1 L481.8,367.4 L483.5,370.2
 L485.3,372.9 L487.0,375.8 L488.7,378.5 L490.4,381.3 L492.1,384.0 L493.9,386.7 L495.6,389.3 L497.3,391.7
 L499.0,394.1 L500.7,399.8 L502.5,402.0 L504.2,404.1 L5

In [ ]:
// Plot2Ddata plt = new Plot2Ddata();

// plt.AddDataGroup(pltTab4_unproc.dataGroups[1]);

// plt.XrangeMin = 0;
// plt.XrangeMax = 2;
// var gp = plt.ToGnuplot();
// gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.95 
 
 
 
 
 0.96 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Wx 
 
 
 
 
 Wx 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M655.1,57.1 L708.5,57.1 M62.2,350.7 L63.9,350.5 L65.6,350.8 L67.4,351.2 L69.1,351.7 L70.8,352.3
 L72.5,353.1 L74.2,354.0 L76.0,355.0 L77.7,356.3 L79.4,357.6 L81.1,359.2 L82.8,360.8 L84.6,362.6
 L86.3,364.2 L88.0,366.4 L89.7,368.6 L91.4,370.7 L93.2,372.9 L94.9,375.2 L96.6,377.4 L98.3,379.7
 L100.0,382.3 L101.8,388.3 L103.5,390.3 L105.2,392.7 L106.9,395.5 L108.6,397.3 L110.4,399.7 L112.1,401.3
 L113.8,403.2 L115.5,365.8 L117.2,364.7 L119.0,363.6 L120.7,362.4 L122.4,361.0 L124.1,359.8 L125.8,358.3
 L127.6,356.9 L129.3,355.6 L131.0,351.0 L132.7,351.5 L134.4,351.7 L136.1,352.3 L137.9,352.4 L139.6,352.6
 L141.3,352.6 L143.0,352.6 L144.7,352.6 L146.5,352.7 L148.2,352.6 L149.9,352.6 L151.6,352.2 L153.3,352.0
 L155.1,352.0 L156.8,352.0 L158.5,352.1 L160.2,344.8 L161.9,345.8 L163.7,346.9 L165.4,348.0 L167.1,353.0
 L168.8,353.6 L170.5,350.5 L172.3,351.9 L174.0,353.2 L175.7,355.8 L177.4,356.6 L179.1,358.2 L180.9,359.3
 L182.6,360.2 L184.3,361.8 L186.0,363.3 L187.7,364.7 L189.5,366.3 L191.2,368.9 L192.9,371.5 L194.6,373.8
 L196.3,377.1 L198.1,379.9 L199.8,382.9 L201.5,385.9 L203.2,388.9 L204.9,391.8 L206.7,394.8 L208.4,397.9
 L210.1,401.0 L211.8,403.7 L213.5,406.7 L215.3,409.6 L217.0,412.4 L218.7,415.2 L220.4,424.8 L222.1,428.3
 L223.9,431.7 L225.6,435.1 L227.3,438.5 L229.0,439.7 L230.7,442.8 L232.5,445.5 L234.2,448.2 L235.9,451.1
 L237.6,453.6 L239.3,456.0 L241.1,458.4 L242.8,460.6 L244.5,462.7 L246.2,465.2 L247.9,467.4 L249.7,469.5
 L251.4,471.4 L253.1,473.1 L254.8,474.7 L256.5,476.3 L258.3,478.0 L260.0,479.1 L261.7,480.5 L263.4,481.3
 L265.1,482.5 L266.9,503.0 L268.6,504.1 L270.3,505.0 L272.0,505.8 L273.7,484.7 L275.4,507.2 L277.2,507.5
 L278.9,484.9 L280.6,507.8 L282.3,507.7 L284.0,507.4 L285.8,507.0 L287.5,506.4 L289.2,505.6 L290.9,504.6
 L292.6,503.5 L294.4,502.3 L296.1,500.9 L297.8,499.4 L299.5,474.6 L301.2,473.4 L303.0,471.9 L304.7,469.8
 L306.4,468.4 L308.1,466.5 L309.8,464.4 L311.6,462.1 L313.3,459.8 L315.0,457.3 L316.7,454.8 L318.4,452.1
 L320.2,449.4 L321.9,446.6 L323.6,443.2 L325.3,440.7 L327.0,437.4 L328.8,434.5 L330.5,431.9 L332.2,429.0
 L333.9,425.9 L335.6,422.7 L337.4,419.4 L339.1,416.1 L340.8,412.8 L342.5,408.7 L344.2,405.7 L346.0,402.3
 L347.7,398.3 L349.4,400.6 L351.1,391.2 L352.8,388.7 L354.6,385.7 L356.3,382.5 L358.0,379.2 L359.7,375.8
 L361.4,372.3 L363.2,368.9 L364.9,357.5 L366.6,354.6 L368.3,351.7 L370.0,348.9 L371.8,346.5 L373.5,343.7
 L375.2,341.2 L376.9,338.5 L378.6,336.3 L380.4,333.7 L382.1,331.0 L383.8,328.5 L385.5,326.0 L387.2,323.6
 L389.0,321.3 L390.7,319.7 L392.4,317.8 L394.1,315.2 L395.8,313.2 L397.6,311.1 L399.3,309.0 L401.0,307.3
 L402.7,305.5 L404.4,303.5 L406.2,301.9 L407.9,300.2 L409.6,298.3 L411.3,296.6 L413.0,295.0 L414.7,293.3
 L416.5,291.8 L418.2,290.4 L419.9,289.2 L421.6,288.2 L423.3,287.6 L425.1,286.8 L426.8,286.0 L428.5,285.2
 L430.2,284.3 L431.9,284.4 L433.7,283.4 L435.4,279.3 L437.1,281.7 L438.8,280.8 L440.5,280.0 L442.3,279.2
 L444.0,278.4 L445.7,274.3 L447.4,273.7 L449.1,273.4 L450.9,275.1 L452.6,274.5 L454.3,273.3 L456.0,272.6
 L457.7,267.4 L459.5,268.3 L461.2,267.5 L462.9,266.8 L464.6,266.0 L466.3,262.1 L468.1,261.1 L469.8,260.0
 L471.5,258.9 L473.2,257.8 L474.9,256.7 L476.7,255.5 L478.4,257.0 L480.1,255.8 L481.8,254.3 L483.5,252.7
 L485.3,251.0 L487.0,247.8 L488.7,246.2 L490.4,244.5 L492.1,242.8 L493.9,241.1 L495.6,239.3 L497.3,237.4
 L499.0,235.6 

In [ ]:
// Plot2Ddata plt = new Plot2Ddata();

// plt.AddDataGroup(pltTab4_unproc.dataGroups[2]);

// plt.XrangeMin = 0;
// plt.XrangeMax = 2;
// var gp = plt.ToGnuplot();
// gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.95 
 
 
 
 
 0.96 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Wy 
 
 
 
 
 Wy 
 
 
 
	<path stroke='rgb(255, 165, 0)' d='M655.1,57.1 L708.5,57.1 M62.2,350.7 L63.9,350.5 L65.6,350.8 L67.4,351.2 L69.1,351.7 L70.8,352.3
 L72.5,353.1 L74.2,354.0 L76.0,355.0 L77.7,356.3 L79.4,357.6 L81.1,359.2 L82.8,360.8 L84.6,362.6
 L86.3,364.2 L88.0,366.4 L89.7,368.6 L91.4,370.8 L93.2,373.0 L94.9,375.2 L96.6,377.5 L98.3,379.7
 L100.0,382.3 L101.8,388.2 L103.5,390.2 L105.2,392.7 L106.9,395.5 L108.6,397.2 L110.4,399.7 L112.1,401.2
 L113.8,403.2 L115.5,365.7 L117.2,364.7 L119.0,363.6 L120.7,362.3 L122.4,361.0 L124.1,359.7 L125.8,358.3
 L127.6,356.8 L129.3,355.6 L131.0,350.9 L132.7,351.5 L134.4,351.7 L136.1,352.3 L137.9,352.5 L139.6,352.6
 L141.3,352.6 L143.0,352.6 L144.7,352.6 L146.5,352.6 L148.2,352.6 L149.9,352.5 L151.6,352.2 L153.3,351.9
 L155.1,352.0 L156.8,352.0 L158.5,352.1 L160.2,344.8 L161.9,345.8 L163.7,346.9 L165.4,348.0 L167.1,353.0
 L168.8,353.6 L170.5,350.4 L172.3,351.9 L174.0,353.3 L175.7,355.9 L177.4,356.6 L179.1,358.2 L180.9,359.4
 L182.6,360.2 L184.3,361.9 L186.0,363.4 L187.7,364.8 L189.5,366.4 L191.2,368.9 L192.9,371.5 L194.6,373.8
 L196.3,377.1 L198.1,380.0 L199.8,383.0 L201.5,385.9 L203.2,388.9 L204.9,391.8 L206.7,394.8 L208.4,397.9
 L210.1,401.0 L211.8,403.7 L213.5,406.7 L215.3,409.5 L217.0,412.4 L218.7,415.2 L220.4,424.8 L222.1,428.3
 L223.9,431.7 L225.6,435.1 L227.3,438.5 L229.0,439.7 L230.7,442.8 L232.5,445.6 L234.2,448.2 L235.9,451.1
 L237.6,453.6 L239.3,456.1 L241.1,458.4 L242.8,460.6 L244.5,462.8 L246.2,465.3 L247.9,467.4 L249.7,469.5
 L251.4,471.4 L253.1,473.1 L254.8,474.8 L256.5,476.3 L258.3,478.0 L260.0,479.1 L261.7,480.5 L263.4,481.3
 L265.1,482.5 L266.9,503.0 L268.6,504.1 L270.3,505.0 L272.0,505.7 L273.7,484.7 L275.4,507.1 L277.2,507.5
 L278.9,484.8 L280.6,507.8 L282.3,507.7 L284.0,507.4 L285.8,507.0 L287.5,506.4 L289.2,505.6 L290.9,504.6
 L292.6,503.6 L294.4,502.3 L296.1,501.0 L297.8,499.4 L299.5,474.6 L301.2,473.4 L303.0,471.9 L304.7,469.8
 L306.4,468.4 L308.1,466.5 L309.8,464.4 L311.6,462.1 L313.3,459.8 L315.0,457.3 L316.7,454.8 L318.4,452.1
 L320.2,449.4 L321.9,446.6 L323.6,443.2 L325.3,440.7 L327.0,437.4 L328.8,434.5 L330.5,431.9 L332.2,429.0
 L333.9,425.9 L335.6,422.7 L337.4,419.5 L339.1,416.2 L340.8,412.8 L342.5,408.7 L344.2,405.7 L346.0,402.4
 L347.7,398.4 L349.4,400.7 L351.1,391.2 L352.8,388.7 L354.6,385.7 L356.3,382.4 L358.0,379.1 L359.7,375.7
 L361.4,372.2 L363.2,368.8 L364.9,357.5 L366.6,354.6 L368.3,351.7 L370.0,348.9 L371.8,346.5 L373.5,343.8
 L375.2,341.2 L376.9,338.5 L378.6,336.3 L380.4,333.6 L382.1,330.9 L383.8,328.4 L385.5,325.9 L387.2,323.5
 L389.0,321.1 L390.7,319.7 L392.4,317.8 L394.1,315.2 L395.8,313.2 L397.6,311.1 L399.3,308.9 L401.0,307.3
 L402.7,305.4 L404.4,303.4 L406.2,301.9 L407.9,300.1 L409.6,298.2 L411.3,296.5 L413.0,294.9 L414.7,293.2
 L416.5,291.7 L418.2,290.4 L419.9,289.2 L421.6,288.2 L423.3,287.6 L425.1,286.8 L426.8,286.0 L428.5,285.2
 L430.2,284.3 L431.9,284.4 L433.7,283.4 L435.4,279.3 L437.1,281.7 L438.8,280.8 L440.5,280.0 L442.3,279.2
 L444.0,278.4 L445.7,274.2 L447.4,273.6 L449.1,273.3 L450.9,275.0 L452.6,271.9 L454.3,273.2 L456.0,272.6
 L457.7,267.3 L459.5,268.3 L461.2,267.5 L462.9,266.7 L464.6,265.9 L466.3,262.1 L468.1,261.0 L469.8,260.0
 L471.5,258.9 L473.2,257.8 L474.9,256.6 L476.7,255.5 L478.4,256.9 L480.1,255.8 L481.8,254.2 L483.5,252.6
 L485.3,250.9 L487.0,247.7 L488.7,246.1 L490.4,244.5 L492.1,242.8 L493.9,241.0 L495.6,239.2 L497.3,237.4
 L499.0,235.

## Case 5 (m3_Oh01_eta03)

In [ ]:
var pltTab5 = PlotTableDM[4,0];
pltTab5.Title

m3_Oh01_eta03

In [ ]:
// var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m3/AspRatioInTime_m3_Oh01_eta03.txt");

// //var dat_plot = new Plot2Ddata();
// var fmt = new PlotFormat();
// fmt.Style = Styles.Lines; 
// fmt.LineWidth = 1.5;
// fmt.LineColor = LineColors.Black;
// pltTab5.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

Error: System.IO.FileNotFoundException: Could not find file 'd:\BoSSS-experimental\public\examples\Oscillating-Droplet\data\LWoverTime\m3\AspRatioInTime_m3_Oh01_eta03.txt'.
File name: 'd:\BoSSS-experimental\public\examples\Oscillating-Droplet\data\LWoverTime\m3\AspRatioInTime_m3_Oh01_eta03.txt'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize)
   at ilPSP.IMatrixExtensions.LoadFromTextFile(String name) in D:\BoSSS-experimental\public\src\ilPSP\layer_1.2-ilPSP\ilPSP\IMatrixExtensions.cs:line 169
   at Submission#241.<<Initialize>>d__0.MoveNext()
--- End of 

In [ ]:
pltTab5.XrangeMin = 0;
pltTab5.XrangeMax = 5;
var gp = pltTab5.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 1.3 
 
 
 
 
 1.35 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta03 
 
 
 BoSSS-J686 
 
 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,93.1 L519.8,93.1 M80.2,448.1 L80.9,448.2 L81.5,447.6 L82.2,446.7 L82.9,445.5 L83.5,443.9
 L84.2,442.2 L84.9,440.2 L85.6,438.2 L86.2,435.9 L86.9,433.6 L87.6,431.8 L88.2,429.4 L88.9,427.3
 L89.6,425.3 L90.2,423.4 L90.9,421.6 L91.6,419.6 L92.3,417.7 L92.9,415.9 L93.6,414.0 L94.3,412.2
 L94.9,410.4 L95.6,408.7 L96.3,406.7 L96.9,405.6 L97.6,403.7 L98.3,401.7 L99.0,400.0 L99.6,398.2
 L100.3,396.4 L101.0,394.2 L101.6,392.7 L102.3,390.7 L103.0,388.9 L103.6,387.7 L104.3,385.9 L105.0,384.6
 L105.7,382.9 L106.3,380.9 L107.0,381.6 L107.7,379.8 L108.3,377.9 L109.0,376.3 L109.7,374.8 L110.3,373.5
 L111.0,372.6 L111.7,371.3 L112.4,372.0 L113.0,370.9 L113.7,370.4 L114.4,369.4 L115.0,368.8 L115.7,368.5
 L116.4,368.1 L117.0,367.7 L117.7,367.4 L118.4,367.1 L119.1,367.0 L119.7,366.8 L120.4,366.7 L121.1,366.7
 L121.7,366.7 L122.4,366.8 L123.1,366.9 L123.7,367.1 L124.4,367.0 L125.1,367.3 L125.8,367.6 L126.4,368.0
 L127.1,368.7 L127.8,369.4 L128.4,370.0 L129.1,370.6 L129.8,370.4 L130.4,371.6 L131.1,372.3 L131.8,373.0
 L132.5,373.6 L133.1,373.8 L133.8,374.1 L134.5,374.7 L135.1,375.4 L135.8,376.2 L136.5,377.2 L137.1,378.3
 L137.8,378.4 L138.5,379.4 L139.2,380.6 L139.8,381.8 L140.5,382.6 L141.2,383.9 L141.8,385.1 L142.5,386.2
 L143.2,387.3 L143.8,388.3 L144.5,389.2 L145.2,390.2 L145.9,391.1 L146.5,390.8 L147.2,390.4 L147.9,391.8
 L148.5,392.3 L149.2,393.9 L149.9,394.4 L150.5,395.2 L151.2,395.6 L151.9,396.2 L152.5,396.8 L153.2,397.2
 L153.9,397.9 L154.6,398.3 L155.2,398.8 L155.9,399.3 L156.6,399.7 L157.2,400.0 L157.9,400.4 L158.6,400.6
 L159.2,400.8 L159.9,400.9 L160.6,401.1 L161.3,401.2 L161.9,401.3 L162.6,401.3 L163.3,401.2 L163.9,401.2
 L164.6,401.1 L165.3,401.0 L165.9,400.9 L166.6,400.8 L167.3,400.6 L168.0,400.5 L168.6,400.3 L169.3,400.1
 L170.0,399.9 L170.6,399.7 L171.3,399.5 L172.0,399.2 L172.6,399.0 L173.3,398.6 L174.0,398.4 L174.7,398.1
 L175.3,397.7 L176.0,397.4 L176.7,397.0 L177.3,396.6 L178.0,396.3 L178.7,395.9 L179.3,395.6 L180.0,395.7
 L180.7,395.1 L181.4,394.7 L182.0,394.3 L182.7,395.0 L183.4,394.8 L184.0,393.9 L184.7,394.8 L185.4,394.8
 L186.0,395.3 L186.7,395.7 L187.4,395.8 L188.1,395.8 L188.7,395.7 L189.4,395.8 L190.1,395.7 L190.7,395.6
 L191.4,395.6 L192.1,395.6 L192.7,395.6 L193.4,395.6 L194.1,396.1 L194.8,396.2 L195.4,396.3 L196.1,396.4
 L196.8,396.6 L197.4,397.1 L198.1,397.4 L198.8,397.4 L199.4,397.3 L200.1,397.4 L200.8,397.5 L201.5,397.7
 L202.1,398.2 L202.8,398.3 L203.5,398.6 L204.1,398.8 L204.8,399.1 L205.5,400.8 L206.1,400.9 L206.8,402.1
 L207.5,403.4 L208.2,404.2 L208.8,405.1 L209.5,405.8 L210.2,406.5 L210.8,407.0 L211.5,407.8 L212.2,408.0
 L212.8,408.7 L213.5,409.2 L214.2,409.3 L214.8,409.9 L215.5,410.4 L216.2,410.9 L216.9,411.4 L217.5,411.8
 L218.2,412.3 L218.9,412.8 L219.5,413.3 L220.2,413.8 L220.9,413.9 L221.5,414.8 L222.2,415.6 L222.9,416.0
 L223.6,416.9 L224.2,417.8 L224.9,418.4 L225.6,419.2 L226.2,419.8 L226.9,420.6 L227.6,421.3 L228.2,421.8
 L228.9,422.4 L229.6,423.1 L230.3,424.1 L230.9,424.7 L231.6,425.4 L232.3,426.1 L232.9,426.8 L233.6,427.8
 L234.3,428.4 L234.9,429.1 L235.6,429.8 L236.3,430.5 L237.0,431.2 L237.6,431.9 L238.3,432.6 L239.0,433.3
 L239.6,434.0 L240.3,434.7 L241.0,435.4 L241.6,436.0 L242.3,4

In [ ]:
foreach(var grp in pltTab5.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m3/" + pltTab5.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 6 (m3_Oh01_eta015)

In [ ]:
var pltTab6 = PlotTableDM[5,0];
pltTab6.Title

m3_Oh01_eta015

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m3/AspRatioInTime_m3_Oh01_eta015.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab6.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab6.XrangeMin = 0;
pltTab6.XrangeMax = 0.5;
var gp = pltTab6.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.995 
 
 
 
 
 1 
 
 
 
 
 1.005 
 
 
 
 
 1.01 
 
 
 
 
 1.015 
 
 
 
 
 1.02 
 
 
 
 
 1.025 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta015 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M88.5,478.5 L95.1,477.5 L101.7,477.1 L108.3,474.5 L115.0,470.4 L121.6,465.0
 L128.2,458.6 L134.8,451.3 L141.4,443.2 L148.0,434.5 L154.7,423.7 L161.3,414.2 L167.9,404.2 L174.5,392.9
 L181.1,382.5 L187.7,371.4 L194.4,360.0 L201.0,348.4 L207.6,336.3 L214.2,325.1 L220.8,315.3 L227.4,303.4
 L234.1,291.6 L240.7,279.9 L247.3,268.3 L253.9,256.8 L260.5,246.1 L267.1,234.6 L273.7,223.9 L280.4,214.1
 L287.0,203.5 L293.6,193.7 L300.2,184.3 L306.8,175.1 L313.4,166.8 L320.1,158.1 L326.7,150.2 L333.3,143.2
 L339.9,135.9 L346.5,129.5 L353.1,123.1 L359.8,117.6 L366.4,112.4 L373.0,107.7 L379.6,103.4 L386.2,99.5
 L392.8,96.0 L399.5,92.7 L406.1,90.0 L412.7,89.3 L419.3,88.1 L425.9,87.2 L432.5,87.6 L439.1,86.9
 L445.8,86.8 L452.4,87.4 L459.0,88.4 L465.6,89.5 L472.2,91.3 L478.8,93.2 L485.5,95.5 L492.1,98.1
 L498.7,101.0 L505.3,104.3 L511.9,107.6 L518.5,111.5 L525.2,115.7 L531.8,120.0 L538.4,124.1 L545.0,127.5
 L551.6,131.0 L558.2,134.7 L564.9,138.5 L571.5,142.4 L578.1,144.9 L584.7,148.9 L591.3,154.0 L597.9,157.2
 L604.5,161.4 L611.2,166.9 L617.8,171.6 L624.4,176.1 L631.0,178.2 L637.6,184.0 L644.2,186.2 L650.9,192.3
 L657.5,194.2 L664.1,200.4 L670.7,205.1 L677.3,209.2 L683.9,208.9 L690.6,215.3 L697.2,219.8 L703.8,223.6
 L710.4,227.0 L717.0,230.1 L723.6,233.1 L730.3,232.3 L736.9,237.4 L743.5,238.0 L750.1,240.2 '/> 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M410.9,117.1 L464.3,117.1 M88.5,478.5 L95.1,468.6 L101.7,457.7 L108.3,447.0 L115.0,436.9 L121.6,427.3
 L128.2,417.9 L134.8,408.8 L141.4,399.8 L148.0,389.9 L154.7,382.5 L161.3,374.2 L167.9,364.7 L174.5,356.5
 L181.1,347.6 L187.7,339.0 L194.4,329.9 L201.0,320.5 L207.6,311.1 L214.2,301.7 L220.8,295.7 L227.4,285.9
 L234.1,276.1 L240.7,266.2 L247.3,256.4 L253.9,246.7 L260.5,237.2 L267.1,228.6 L273.7,218.8 L280.4,209.9
 L287.0,201.8 L293.6,192.7 L300.2,184.7 L306.8,176.9 L313.4,169.4 L320.1,161.8 L326.7,154.4 L333.3,148.1
 L339.9,142.2 L346.5,136.1 L353.1,130.9 L359.8,125.8 L366.4,121.1 L373.0,117.1 L379.6,113.3 L386.2,109.6
 L392.8,106.2 L399.5,103.1 L406.1,99.6 L412.7,97.3 L419.3,95.8 L425.9,96.3 L432.5,95.7 L439.1,95.3
 L445.8,95.8 L452.4,95.0 L459.0,96.3 L465.6,98.2 L472.2,99.8 L478.8,102.0 L485.5,104.0 L492.1,106.6
 L498.7,109.0 L505.3,111.9 L511.9,114.6 L518.5,117.7 L525.2,121.0 L531.8,124.9 L538.4,128.9 L545.0,133.0
 L551.6,136.9 L558.2,140.0 L564.9,143.3 L571.5,146.7 L578.1,150.1 L584.7,153.7 L591.3,155.4 L597.9,160.1
 L604.5,164.5 L611.2,166.7 L617.8,170.7 L624.4,176.2 L631.0,181.0 L637.6,185.5 L644.2,186.9 L650.9,193.2
 L657.5,198.2 L664.1,198.8 L670.7,205.5 L677.3,206.5 L683.9,213.5 L690.6,218.6 L697.2,223.0 L703.8,221.2
 L710.4,228.5 L717.0,233.6 L723.6,237.7 L730.3,241.4 L736.9,244.8 L743.5,247.9 L750.1,250.8 '/> 
 
 BoSSS-J686-AMR2 
 
 
 BoSSS-J686-AMR2 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray=' 8,4,2,4' d='M410.9,141.1 L464.3,141.1 M88.5,478.5 L91.8,458.6 L95.1,458.8 L98.4,458.9 L101.7,458.5 L105.0,457.6
 L108.3,456.3 L111.7,455.0 L115.0,452.7 L118.3,450.0 L121.6,447.0 L124.9,443.8 L128.2,440.4 L131.5,436.8
 L134.8,433.0 L138.1,429.9 L141.4,425.7 L144.7,421.0 L148.0,416.2 L151.4,412.1 L154.7,407.3 L158.0,403.9
 L161.3,398.8 L164.6,393.5 L167.9,389.4 L171.2,38

In [ ]:
foreach(var grp in pltTab6.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m3/" + pltTab6.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 7 (m4_Oh01_eta04)

In [ ]:
var pltTab7 = PlotTableDM[6,0];
pltTab7.Title

m4_Oh01_eta04

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh01_eta04.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab7.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab7.XrangeMin = 0;
pltTab7.XrangeMax = 7;
var gp = pltTab7.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 1.3 
 
 
 
 
 1.35 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta04 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M80.2,207.1 L80.7,207.1 L81.2,207.1 L81.6,207.0 L82.1,206.9 L82.6,206.9
 L83.1,206.8 L83.5,206.7 L84.0,206.5 L84.5,206.4 L85.0,206.3 L85.5,206.0 L85.9,205.8 L86.4,205.5
 L86.9,205.2 L87.4,204.8 L87.9,204.5 L88.3,204.2 L88.8,203.8 L89.3,203.4 L89.8,203.1 L90.2,202.7
 L90.7,202.5 L91.2,202.0 L91.7,201.5 L92.2,201.0 L92.6,200.5 L93.1,200.1 L93.6,199.6 L94.1,199.2
 L94.6,198.2 L95.0,197.7 L95.5,197.1 L96.0,196.6 L96.5,196.0 L96.9,195.7 L97.4,195.2 L97.9,195.1
 L98.4,195.0 L98.9,194.9 L99.3,194.7 L99.8,195.0 L100.3,195.5 L100.8,196.3 L101.3,197.4 L101.7,198.8
 L102.2,200.5 L102.7,202.5 L103.2,204.7 L103.6,207.2 L104.1,210.0 L104.6,213.1 L105.1,216.7 L105.6,220.7
 L106.0,225.0 L106.5,229.8 L107.0,235.1 L107.5,240.7 L108.0,246.7 L108.4,252.6 L108.9,258.7 L109.4,265.7
 L109.9,273.3 L110.3,281.5 L110.8,290.5 L111.3,298.4 L111.8,304.8 L112.3,312.2 L112.7,320.0 L113.2,328.3
 L113.7,337.1 L114.2,346.1 L114.7,354.9 L115.1,363.5 L115.6,371.6 L116.1,379.3 L116.6,386.5 L117.0,393.2
 L117.5,398.9 L118.0,404.7 L118.5,410.0 L119.0,415.0 L119.4,419.6 L119.9,423.8 L120.4,427.3 L120.9,430.5
 L121.4,433.3 L121.8,435.7 L122.3,437.5 L122.8,438.9 L123.3,439.7 L123.7,440.1 L124.2,439.9 L124.7,439.3
 L125.2,438.2 L125.7,436.7 L126.1,434.9 L126.6,432.7 L127.1,430.2 L127.6,427.4 L128.1,424.4 L128.5,421.2
 L129.0,417.8 L129.5,413.5 L130.0,409.5 L130.4,405.6 L131.4,397.4 L131.9,393.5 L132.4,389.6 L132.8,385.8
 L133.3,382.0 L133.8,378.3 L135.2,365.7 L135.7,362.2 L136.2,358.9 L136.7,355.5 L137.1,352.2 L137.6,348.9
 L138.1,345.7 L138.6,342.5 L139.1,339.3 L139.5,336.2 L140.0,330.5 L140.5,327.7 L141.0,324.5 L141.4,321.7
 L141.9,319.0 L142.4,315.9 L142.9,312.9 L143.4,310.1 L143.8,307.4 L144.3,304.8 L144.8,308.8 L145.3,306.1
 L145.8,303.6 L146.2,301.6 L146.7,299.9 L147.2,301.7 L148.1,299.4 L149.1,297.4 L149.6,296.4 L150.1,294.9
 L150.5,293.4 L151.0,291.9 L151.5,290.9 L152.0,290.0 L152.5,290.4 L152.9,290.3 L153.4,290.3 L153.9,290.3
 L154.4,290.5 L154.8,290.7 L155.3,293.9 L155.8,294.1 L156.3,294.2 L156.8,294.4 L157.2,294.3 L157.7,294.2
 L158.2,294.1 L158.7,294.2 L159.2,294.4 L159.6,295.2 L160.1,295.6 L160.6,296.2 L161.1,296.8 L161.5,297.4
 L162.0,298.1 L162.5,298.9 L163.0,299.7 L163.5,300.6 L163.9,301.5 L164.4,302.4 L164.9,303.4 L165.4,304.4
 L165.9,305.5 L166.3,306.7 L166.8,307.8 L167.3,309.1 L167.8,310.3 L168.2,311.7 L168.7,313.0 L169.2,314.4
 L169.7,315.9 L170.2,317.4 L170.6,318.9 L171.1,320.4 L171.6,322.0 L172.1,323.7 L172.6,325.8 L173.0,327.9
 L173.5,332.8 L174.0,335.1 L174.5,337.0 L174.9,338.5 L175.4,340.2 L175.9,342.0 L176.4,344.8 L176.9,346.9
 L177.8,350.9 L178.3,352.9 L178.8,355.0 L179.2,357.2 L179.7,359.5 L180.2,362.2 L180.7,364.9 L181.2,369.7
 L181.6,372.2 L182.1,374.4 L182.6,376.8 L183.1,379.3 L183.6,381.9 L184.0,384.6 L184.5,387.4 L185.0,390.2
 L185.5,393.0 L185.9,395.8 L186.4,398.7 L186.9,401.5 L187.4,404.3 L187.9,407.0 L188.3,409.8 L188.8,412.5
 L189.3,415.1 L189.8,417.8 L190.3,420.4 L190.7,423.0 L191.2,426.8 L191.7,429.4 L192.2,432.3 L192.6,434.5
 L193.1,437.3 L193.6,439.8 L194.1,442.2 L194.6,444.4 L195.0,446.6 L195.5,448.7 L196.0,450.7 L196.5,452.7
 L197.0,454.8 L197.4,456.8 L197.9,458.7 L19

In [ ]:
foreach(var grp in pltTab7.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m4/" + pltTab7.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 8 (m4_Oh01_eta01)

In [ ]:
var pltTab8 = PlotTableDM[7,0];
pltTab8.Title

m4_Oh01_eta01

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh01_eta01.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab8.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab8.XrangeMin = 0;
pltTab8.XrangeMax = 1;
var gp = pltTab8.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 1.05 
 
 
 
 
 1.06 
 
 
 
 
 1.07 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta01 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M80.2,117.6 L83.5,117.6 L86.9,117.9 L90.2,118.4 L93.6,119.2 L96.9,120.1
 L100.3,121.1 L103.6,122.5 L107.0,124.2 L110.3,126.0 L113.7,128.1 L117.0,130.4 L120.4,132.9 L123.7,135.7
 L127.1,138.3 L130.4,141.5 L133.8,145.0 L137.1,148.6 L140.5,152.5 L143.8,156.6 L147.2,160.9 L150.5,165.4
 L153.9,170.2 L157.2,175.2 L160.6,180.8 L163.9,186.5 L167.3,192.1 L170.6,198.3 L174.0,204.7 L177.3,211.3
 L180.7,218.2 L184.0,225.2 L187.4,232.4 L190.7,239.9 L194.1,247.5 L197.4,255.3 L200.8,263.2 L204.1,271.3
 L207.5,279.6 L210.8,288.0 L214.2,296.5 L217.5,305.0 L220.9,313.7 L224.2,322.4 L227.6,331.1 L230.9,339.8
 L234.3,348.4 L237.6,357.1 L241.0,365.7 L244.3,374.2 L247.7,382.3 L251.0,390.6 L254.4,398.9 L257.7,406.8
 L261.1,414.7 L264.4,422.4 L267.8,429.9 L271.1,437.1 L274.5,444.2 L277.8,450.9 L281.2,457.4 L284.5,463.6
 L287.9,469.1 L291.2,474.8 L294.6,480.2 L297.9,485.3 L301.3,490.0 L304.6,494.4 L308.0,498.4 L311.3,502.1
 L314.7,505.5 L318.0,508.5 L321.4,511.1 L324.7,513.4 L328.1,514.9 L331.4,516.5 L334.8,517.9 L338.1,519.0
 L341.5,519.7 L344.8,520.0 L348.2,520.0 L351.5,519.8 L354.9,519.1 L358.2,518.2 L361.6,517.0 L364.9,515.6
 L368.3,513.8 L371.6,511.9 L375.0,509.6 L378.3,506.8 L381.7,504.3 L385.0,501.6 L388.4,498.7 L391.7,495.5
 L395.1,492.2 L398.4,488.8 L401.8,485.2 L405.1,481.4 L408.5,477.6 L411.8,473.6 L415.2,469.6 L418.5,465.5
 L421.8,461.3 L425.2,456.5 L428.5,452.4 L431.9,448.2 L435.2,444.0 L438.6,439.7 L441.9,435.4 L445.3,431.1
 L448.6,426.7 L452.0,422.4 L455.3,418.1 L458.7,413.8 L462.0,409.5 L465.4,405.3 L468.7,401.1 L472.1,396.9
 L475.4,392.2 L478.8,388.4 L482.1,384.6 L485.5,380.8 L488.8,376.5 L492.2,373.1 L495.5,369.7 L498.9,366.3
 L502.2,362.9 L505.6,359.6 L508.9,356.4 L512.3,353.3 L515.6,350.2 L519.0,347.2 L522.3,344.2 L525.7,341.4
 L529.0,338.7 L532.4,336.1 L535.7,333.6 L539.1,331.2 L542.4,328.9 L545.8,326.7 L549.1,324.6 L552.5,322.7
 L555.8,320.8 L559.2,319.1 L562.5,317.5 L565.9,316.0 L569.2,314.6 L572.6,313.3 L575.9,312.1 L579.3,311.0
 L582.6,310.1 L586.0,309.2 L589.3,308.4 L592.7,307.8 L596.0,307.2 L599.4,306.8 L602.7,306.5 L606.1,306.2
 L609.4,306.1 L612.8,306.1 L616.1,306.1 L619.5,306.3 L622.8,306.6 L626.2,306.9 L629.5,307.4 L632.9,308.0
 L636.2,308.6 L639.6,309.3 L642.9,310.2 L646.3,311.1 L649.6,312.1 L653.0,313.2 L656.3,314.4 L659.7,315.6
 L663.0,316.9 L669.7,319.8 L673.1,321.4 L676.4,323.0 L679.8,324.7 L683.1,326.4 L686.5,328.3 L689.8,330.1
 L693.2,332.1 L696.5,334.1 L699.9,336.1 L703.2,338.2 L706.6,340.3 L709.9,342.5 L713.3,344.7 L716.6,346.9
 L720.0,349.2 L723.3,351.5 L726.7,353.8 L730.0,356.2 L733.4,358.6 L736.7,360.9 L740.1,363.3 L743.4,365.7
 L746.8,368.2 L750.1,370.6 '/> 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 BoSSS-J686-AMR1-3OrdInit 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M410.9,117.1 L464.3,117.1 M80.2,117.6 L83.5,114.6 L86.9,111.9 L90.2,110.3 L93.6,109.7 L96.9,109.9
 L100.3,110.4 L103.6,111.8 L107.0,113.7 L110.3,116.0 L113.7,118.4 L117.0,121.1 L120.4,124.0 L123.7,127.0
 L127.1,129.8 L130.4,133.3 L133.8,137.0 L137.1,140.8 L140.5,144.7 L143.8,148.8 L147.2,153.1 L150.5,157.5
 L153.9,162.2 L157.2,167.0 L160.6,172.0 L163.9,177.4 L167.3,183.4 L170.6,189.4 L174.0,195.3 L177.3,

In [ ]:
foreach(var grp in pltTab8.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m4/" + pltTab8.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 9 (m4_Oh056_eta005)

In [ ]:
var pltTab9 = PlotTableDM[8,0];
pltTab9.Title

m4_Oh056_eta005

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh056_eta005.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 2;
fmt.LineColor = LineColors.Black;
fmt.DashType = DashTypes.Dashed;
pltTab9.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab9.XrangeMin = 0;
pltTab9.XrangeMax = 4;
var gp = pltTab9.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h056 e ta005 
 
 
 BoSSS-J686-AMR1 
 
 
 
 
 BoSSS-J686-AMR1 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,93.1 L464.3,93.1 M80.2,134.8 L81.0,134.9 L81.9,135.1 L82.7,135.5 L83.5,136.2 L84.4,137.1
 L85.2,138.2 L86.1,139.4 L86.9,140.8 L87.7,142.3 L88.6,143.9 L89.4,145.7 L90.2,147.6 L91.1,149.6
 L91.9,151.7 L92.8,153.8 L93.6,156.0 L94.4,158.3 L95.3,160.7 L96.1,163.1 L96.9,165.5 L97.8,168.0
 L98.6,170.5 L99.5,173.1 L100.3,175.6 L101.1,178.2 L102.0,180.8 L102.8,183.4 L103.6,186.0 L104.5,188.7
 L105.3,191.3 L106.2,193.9 L107.0,196.5 L107.8,199.1 L108.7,201.7 L109.5,204.3 L110.3,206.8 L111.2,209.4
 L112.0,211.9 L112.9,214.4 L113.7,216.9 L114.5,219.3 L115.4,221.8 L116.2,224.2 L117.0,226.5 L117.9,228.9
 L118.7,231.2 L119.6,233.5 L120.4,235.7 L121.2,237.9 L122.1,240.1 L122.9,242.3 L123.7,244.4 L124.6,246.5
 L125.4,248.5 L126.3,250.5 L127.1,252.5 L127.9,254.5 L128.8,256.4 L129.6,258.2 L130.4,260.1 L131.3,261.9
 L132.1,263.7 L133.0,265.4 L133.8,267.1 L134.6,268.8 L135.5,270.4 L136.3,272.0 L137.1,273.6 L138.0,275.2
 L138.8,276.7 L139.7,278.2 L140.5,279.6 L141.3,281.0 L142.2,282.4 L143.0,283.8 L143.8,285.1 L144.7,286.4
 L145.5,287.7 L146.4,288.9 L147.2,290.1 L148.0,291.3 L148.9,292.5 L149.7,293.6 L150.5,294.7 L151.4,295.8
 L152.2,296.8 L153.1,297.9 L153.9,298.9 L154.7,299.9 L155.6,300.8 L156.4,301.7 L157.2,302.7 L158.1,303.6
 L158.9,304.4 L159.8,305.3 L160.6,306.1 L161.4,306.9 L162.3,307.7 L163.1,308.5 L163.9,309.2 L164.8,310.0
 L165.6,310.7 L166.4,311.4 L167.3,312.1 L168.1,312.7 L169.0,313.4 L169.8,314.0 L170.6,314.6 L171.5,315.2
 L172.3,315.8 L173.1,316.4 L174.0,317.0 L174.8,317.5 L175.7,318.0 L176.5,318.5 L177.3,319.1 L178.2,319.5
 L179.0,320.0 L179.8,320.5 L180.7,320.9 L181.5,321.4 L182.4,321.8 L183.2,322.3 L184.0,322.7 L184.9,323.1
 L185.7,323.5 L186.5,323.8 L187.4,324.2 L188.2,324.6 L189.1,324.9 L189.9,325.3 L190.7,325.6 L191.6,326.0
 L192.4,326.3 L193.2,326.6 L194.1,326.9 L194.9,327.2 L195.8,327.5 L196.6,327.8 L197.4,328.1 L198.3,328.3
 L199.1,328.6 L199.9,328.9 L200.8,329.1 L201.6,329.4 L202.5,329.6 L203.3,329.8 L204.1,330.1 L205.0,330.3
 L205.8,330.5 L206.6,330.7 L207.5,330.9 L208.3,331.1 L209.2,331.3 L210.0,331.5 L210.8,331.7 L211.7,331.9
 L212.5,332.1 L213.3,332.3 L214.2,332.4 L215.0,332.6 L215.9,332.8 L216.7,332.9 L217.5,333.1 L218.4,333.2
 L219.2,333.4 L220.0,333.5 L220.9,333.7 L221.7,333.8 L222.6,334.0 L223.4,334.1 L224.2,334.2 L225.1,334.4
 L225.9,334.5 L226.7,334.6 L227.6,334.7 L228.4,334.8 L229.3,335.0 L230.1,335.1 L230.9,335.2 L231.8,335.3
 L232.6,335.4 L233.4,335.5 L234.3,335.6 L235.1,335.7 L236.0,335.8 L236.8,335.9 L237.6,336.0 L238.5,336.1
 L239.3,336.2 L240.1,336.3 L241.0,336.4 L241.8,336.4 L242.7,336.5 L243.5,336.6 L244.3,336.7 L245.2,336.8
 L246.0,336.8 L246.8,336.9 L247.7,337.0 L248.5,337.1 L249.3,337.1 L250.2,337.2 L251.0,337.3 L251.9,337.3
 L252.7,337.4 L253.5,337.5 L254.4,337.5 L255.2,337.6 L256.0,337.7 L256.9,337.7 L257.7,337.8 L258.6,337.8
 L259.4,337.9 L260.2,338.0 L261.1,338.0 L261.9,338.1 L262.7,338.1 L263.6,338.2 L264.4,338.2 L265.3,338.3
 L266.1,338.3 L266.9,338.4 L267.8,338.4 L268.6,338.5 L269.4,338.5 L270.3,338.6 L271.1,338.6 L272.0,338.7
 L272.8,338.7 L273.6,338.7 L274.5,338.8 L275.3,338.8 L276.1,338.9 L277.0,338.9 L277.8,338.9 L278.7,339.0
 L279.5,339.0 L280.3,339.1 L281.2,339.1

In [ ]:
foreach(var grp in pltTab9.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m4/" + pltTab9.Title + "_" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}